<a href="https://colab.research.google.com/github/MostafaJahanian/Free_Projects/blob/main/CIFAR10_Dataset_Classification_Using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import random

### Loading Dataset From Keras 

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [3]:
train_images.shape, test_images.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [5]:
len(np.unique(train_labels))

10

In [7]:
# copied from keras api
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog' ,'frog', 'horse', 'ship' ,'truck']

### Getting Data Ready for Training

In [9]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size = 0.1, shuffle = True, random_state = 1)
X_train.shape, X_val.shape

((45000, 32, 32, 3), (5000, 32, 32, 3))